# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import GeoApify API key
import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
geoapify_key = os.getenv("geoapify_key")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,1.90,83,52,2.68,CA,1712862486
1,1,albany,42.6001,-73.9662,14.21,80,100,1.79,US,1712862002
2,2,uturoa,-16.7333,-151.4333,28.56,79,19,7.43,PF,1712862384
3,3,bethel,41.3712,-73.4140,15.20,87,100,1.54,US,1712862101
4,4,antsiranana,-12.2787,49.2917,25.61,91,7,1.57,MG,1712862480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
cities_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map
cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Cities with wind speed less than equal to 0.5 m/s
ideal_df = city_data_df.loc[city_data_df["Wind Speed"] <= 0.50]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bilibino,68.0546,166.4372,-18.64,99,100,0.49,RU,1712862373
29,29,susuman,62.7833,148.1667,-20.01,100,99,0.45,RU,1712862498
42,42,howrah,22.5892,88.3103,27.00,61,40,0.00,IN,1712862503
57,57,severo-yeniseyskiy,60.3725,93.0408,-19.68,99,93,0.45,RU,1712862510
70,70,luebo,-5.3500,21.4167,25.09,80,100,0.34,CD,1712862515
72,72,kalemie,-5.9475,29.1947,22.93,83,95,0.20,CD,1712862516
86,86,aripuana,-9.1667,-60.6333,28.67,90,100,0.48,BR,1712862241
128,128,murun-kuren,49.6342,100.1625,1.76,89,100,0.34,MN,1712862541
138,138,lincoln city,44.9582,-124.0179,13.96,84,100,0.45,US,1712862545
144,144,mawu,29.5923,107.3139,14.19,93,95,0.50,CN,1712862546


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bilibino,RU,68.0546,166.4372,99,
29,susuman,RU,62.7833,148.1667,100,
42,howrah,IN,22.5892,88.3103,61,
57,severo-yeniseyskiy,RU,60.3725,93.0408,99,
70,luebo,CD,-5.3500,21.4167,80,
72,kalemie,CD,-5.9475,29.1947,83,
86,aripuana,BR,-9.1667,-60.6333,90,
128,murun-kuren,MN,49.6342,100.1625,89,
138,lincoln city,US,44.9582,-124.0179,84,
144,mawu,CN,29.5923,107.3139,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bilibino - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
severo-yeniseyskiy - nearest hotel: Актолик
luebo - nearest hotel: No hotel found
kalemie - nearest hotel: Hotel du Lac
aripuana - nearest hotel: No hotel found
murun-kuren - nearest hotel: No hotel found
lincoln city - nearest hotel: Inn at Lincoln City
mawu - nearest hotel: No hotel found
chita - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
butterworth - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA
ngaoundere - nearest hotel: LA CONCORDE
turpan - nearest hotel: Oriental
san luis - nearest hotel: Hotel Dos Venados
walvis bay - nearest hotel: Atlantic
wanning - nearest hotel: No hotel found
mantua - nearest hotel: No hotel found
mir bachah kot - nearest hotel: No hotel found
san kamphaeng - nearest hotel: Viewdoi Art & Resort
crystal city - nearest hotel: No hotel found
ruteng - 

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bilibino,RU,68.0546,166.4372,99,No hotel found
29,susuman,RU,62.7833,148.1667,100,No hotel found
42,howrah,IN,22.5892,88.3103,61,Sun India Guest House
57,severo-yeniseyskiy,RU,60.3725,93.0408,99,Актолик
70,luebo,CD,-5.3500,21.4167,80,No hotel found
72,kalemie,CD,-5.9475,29.1947,83,Hotel du Lac
86,aripuana,BR,-9.1667,-60.6333,90,No hotel found
128,murun-kuren,MN,49.6342,100.1625,89,No hotel found
138,lincoln city,US,44.9582,-124.0179,84,Inn at Lincoln City
144,mawu,CN,29.5923,107.3139,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)